### All important imports and var defs

In [198]:
import json

In [199]:
import pickle
with open('final_vocab.pickle', 'rb') as file:
    loaded_data = pickle.load(file)
    print(len(loaded_data))

395515


In [200]:
import nltk
from nltk.corpus import stopwords, words
 
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# set of common words that are often considered unimportant, such as "the", "is", "in", "and", etc.

nltk.download('words')
word_set = set(words.words())

with open('mesh_vocab.pickle', 'rb') as file:
    loaded_mesh_vocab = pickle.load(file) # contains 1409 entities related to medical subject headings

with open('nif_vocab_non_nltk_words.pickle', 'rb') as file: # contains specialized vocabulary terms related to neuroscience that are not included in the standard NLTK word list.
    loaded_nif_vocab_non_nltk_words = pickle.load(file) # length=3,90,880

with open('nif_vocab_nltk_words_cleaned.pickle', 'rb') as file: # Common English words which are interesting for us, e.g. neuron, axon etc
    loaded_nltk_words_cleaned = pickle.load(file) # length=3844

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/users/shubham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/users/shubham/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [201]:
# len(word_set), len(stop_words) #(235892, 179)
# len(loaded_nltk_words_cleaned) # 3844

### Helper functions

In [202]:
def remove_punctuation(text):
    # Define punctuation to be removed (excluding +, -, =, and parentheses)
    custom_punctuation = string.punctuation.replace("+", "").replace("-", "").replace("=>", "").replace("=", "").replace("(", "").replace(")", "")
    
    # Tokenize the text
    tokens = text.split()    
    # Remove stopwords and strip unwanted punctuation
    tokens = [w.strip(custom_punctuation) for w in tokens if w not in stop_words]
    return tokens

In [203]:
import pandas as pd
def get_entity_df(text):
    entities = find_entity(text, dictionary, 90)
    ents_dict = entities['entities']
    ents = [k['entity'] for k in ents_dict]
    
    # df = pd.DataFrame(ents_dict)[['entity','similar_with','category','score']]
    df = pd.DataFrame(ents_dict)
    return df

In [204]:
def remove_unmatched_brackets(text):
    stack = []
    result = []

    # Dictionary to match brackets
    bracket_pairs = {')': '(', ']': '[', '}': '{'}
    
    for char in text:
        if char in '({[':
            stack.append(char)
            result.append(char)
        elif char in ')}]':
            if stack and stack[-1] == bracket_pairs[char]:
                stack.pop()
                result.append(char)
            # Skip unmatched closing brackets
        else:
            result.append(char)
    
    # Remove unmatched opening brackets from result
    while stack:
        unmatched = stack.pop()
        result.remove(unmatched)
    return ''.join(result)


In [205]:
irrelevant_single_chars = [")","","="," ","(","{","}","[","]","<",">"]
def clean_entities(entity_list):
    entities = [k for k in entity_list if k.strip() not in irrelevant_single_chars]
    cleaned_entities = [remove_unmatched_brackets(entity.strip()).encode('ascii', 'ignore').decode('ascii') for entity in entities]
    cleaned_entities = [entity for entity in cleaned_entities if entity]
    return cleaned_entities

In [206]:
# https://github.com/hanifabd/lexifuzz-ner
def get_entities_from_lexifuzz(text):
    entities = find_entity(text, dictionary, 90)
    ents_dict = entities['entities']
    ents = [k['entity'] for k in ents_dict]
    ents = clean_entities(ents)
    return ents
# text = "applied the β2* nAChR antagonist DhβE (10 μm, IC90) during FSCV assays (E,F). Prior research has shown that DhβE decreases dorsal striatal DA release by 50% (0.1 μm; ) and NAc core DA release by 80% (1 μm; ).  to PGVG and GA-mix noncontingent paradigm as described above. examined NAc core DA release following 5 Hz and 60 Hz stimulations in the absence and presence of DhβE. effect of DhβE (F(1,35) = 107, p < 0.0001) but no significance for factors of drug/stimulation (F(3,35) = 0.82, p = 0.49) or interaction (F(3,35) = 0.85, p = 0.48) in male mice. We also detected a significant effect of DhβE (F(1,25) = 197, p < 0.0001) but not drug/stimulation (F(3,25) = 0.24, p = 0.87) or interaction (F(3,25) = 0.47, p = 0.71) in female mice."
# get_entities_from_lexifuzz(text)

In [207]:
def get_paper_ids_to_process(number=0):
    with open("NER_papers_processed.txt",'r') as f:
        number_of_papers_processed_till_now = len(f.readlines())
        
    with open("neuro_IDs_for_neuroscience.txt",'r') as f:
        paper_list = f.readlines()
        start = number_of_papers_processed_till_now
        end = number_of_papers_processed_till_now + number
        result = [k.strip() for k in paper_list[start:end]]
    return result
# get_paper_ids_to_process(5)

### lexifuzz_ner new code

In [208]:
import re, string
from thefuzz import fuzz
from thefuzz import process

import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from uuid import uuid4
import sys, time

#### function getFuzzySimilarity

In [209]:
def getFuzzySimilarity(token=None, dictionary=None, min_ratio=None):
    """
    This function uses the TheFuzz library to find the highest matching similary score of a token in a list of values.
    We then compare the similary score of the fuzzy match with our minimum threshold and return a match if the match > treshold.

    Source: https://medium.com/tailo-ai/fuzzy-matching-entities-in-a-custom-entity-dictionary-310158d2b60e
    """

    # Check for appropriate formats
    assert isinstance(token, str), "Tokens can be str() type only"
    assert isinstance(dictionary, dict), "Dictionary format should be provided in the dictionary parameter."
    assert isinstance(min_ratio, int), "Integer format should be provided in the minimum-ratio parameter."

    for key, values in dictionary.items():
        # Using the process option of FuzzyWuzzy, we can search through the entire dictionary for the best match
        match = process.extractOne(token, values, scorer = fuzz.ratio)

        # Match is a tuple with the match value and the similary score.
        if min_ratio <= match[1]:
            return (match + (key, ))

#### function handle_slicing

In [210]:
def handle_slicing(data=None):
  """
    This function takes a dictionary data as input and processes its 'entities' by sorting them based on their score in descending order.
    It then identifies entities with the highest scores, ensuring there is no overlap in their index ranges.
    The function returns the modified data dictionary with the selected high-scoring entities, effectively handling slicing and overlapping index ranges.
  """

  assert isinstance(data, dict), "Dictionary format should be provided in the dictionary parameter."

  # Sort entities by their score in descending order
  sorted_entities = sorted(data['entities'], key=lambda x: (-len(x['entity']), x['index']['start'], -x['index']['end']))

  # Initialize a dictionary to keep track of which indices have been covered
  indices_covered = set()

  # Create a new list of entities with the highest score and ensure no overlap
  new_entities = []
  for entity in sorted_entities:
      start = entity['index']['start']
      end = entity['index']['end']

      # Check if the entity's indices overlap with previously covered indices
      if all(start > end_covered or end < start_covered for start_covered, end_covered in indices_covered):
          new_entities.append(entity)
          # Update the covered indices
          indices_covered.add((start, end))

  # Update the entities in the data dictionary
  data['entities'] = sorted(new_entities, key=lambda x: (x['index']['start']))
  return data

#### function get_actual_detected_entity

In [211]:
def get_actual_detected_entity(actual_ent, detected_words):
    actual_ent_list = actual_ent.split() # with stopwords
    detected_words_list = detected_words.split()
    recreated_str_list = [None]*len(actual_ent_list)
    for i, word in enumerate(actual_ent_list):
        if word in stop_words:
            recreated_str_list[i] = word
    #print(f"{recreated_str_list}, {actual_ent}, {detected_words}")
    cur_detected_index = 0
    for i in range(len(recreated_str_list)):
        if recreated_str_list[i] is None:
            if cur_detected_index < len(detected_words_list): #actual_ent has words which are not stop words, but not present in detected_words
                recreated_str_list[i] = detected_words_list[cur_detected_index]
                cur_detected_index += 1
    
    return ' '.join([w for w in recreated_str_list if w is not None])

#### function get_actual_text_index

In [212]:
def get_actual_text_index(similar_with, compared_text, current_index, text):
    actual_ent_list = vocab_change_dict[similar_with]
    if len(actual_ent_list) == 1:
        actual_ent = actual_ent_list[0]
        actual_entity_in_text = get_actual_detected_entity(actual_ent, compared_text)
        if actual_entity_in_text is not None:
            # Find the start and end indices correctly using current_index
            text_index = text.find(actual_entity_in_text, current_index)
            if text_index >= 0:
                return (text_index, actual_entity_in_text, actual_ent)
    else:
        start_index = sys.maxsize
        return_text = None
        return_actual_ent = None
        for actual_ent in actual_ent_list: #Find the actual entity whch starts first from current_index
            actual_entity_in_text = get_actual_detected_entity(actual_ent, compared_text)
            if actual_entity_in_text is None:
                continue
            cur_start_index = text.find(actual_entity_in_text, current_index)
            if cur_start_index >= 0 and cur_start_index < start_index:
                start_index = cur_start_index
                return_text = actual_entity_in_text
                return_actual_ent = actual_ent

        if start_index != sys.maxsize:
           return (start_index, return_text, return_actual_ent)
    
    return (None, None)

#### function find_entity

In [213]:
def find_entity(text=None, dictionary=None, min_ratio=None):
  """
    This function employs a fuzzy matching algorithm to identify entities in a given text.
    It utilizes a provided dictionary of phrases, comparing them with the text to find matches with a similarity score above a specified threshold.
    The function then returns the detected entities along with their categories, scores, and index positions.
    Additionally, it performs further processing by removing overlapping entities and annotating the original text with custom tags.
  """

  assert isinstance(text, str), "Tokens can be str() type only"
  assert isinstance(dictionary, dict), "Dictionary format should be provided in the dictionary parameter."
  assert isinstance(min_ratio, int), "Integer format should be provided in the minimum-ratio parameter."

  trans_table = str.maketrans('', '', string.punctuation)
  verb_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'] 
    
  result_detection = {}
  result_detection['entities'] = []
  tokens = remove_punctuation(text)  # CHANGED HERE FOR PUNCTUATION
  max_ngrams = max([max(len(phrase.split()) for phrase in phrases) for phrases in dictionary.values()])
  for n in range(1, max_ngrams+1):
    ngrams_result = list(ngrams(tokens, n))
    current_index = 0
    for i, result in enumerate(ngrams_result):
      compared_text = ' '.join(result)
      #compared_text = compared_text.translate(trans_table) #Remove punctuation
      if (compared_text in word_set) and (compared_text not in loaded_nltk_words_cleaned): #Remove non-interesting common english words
          continue
      if n == 1:
          pos_result = nltk.pos_tag([compared_text])
          pos_tag_type = pos_result[0][1]
          if pos_tag_type in verb_pos_tags:
              # print(f'Skip verb {compared_text}')
              continue
      similarity_score = getFuzzySimilarity(token = compared_text, dictionary = dictionary, min_ratio = min_ratio)
      start_index = -1
      if not similarity_score == None: 
        if len(compared_text)<=5 and similarity_score[1]<100: # TASK 1
            continue
        similar_with = similarity_score[0]
        
        
        if similar_with in vocab_change_dict:
            #print(similar_with)
            actual_text_index_result = get_actual_text_index(similar_with, compared_text, current_index, text)
            #print(actual_text_index_result)
            if actual_text_index_result[0] is not None:
                start_index = actual_text_index_result[0]
                compared_text = actual_text_index_result[1]
                similar_with = actual_text_index_result[2]
        else:
            start_index = text.find(compared_text, current_index)
      
      if start_index >= 0:
        #   start_index = 0
        end_index = start_index + len(compared_text) - 1

        # Update current_index to start searching for the next occurrence after the current one. For ngram > 1, current_index will be back to the second index of the ngram value
        #if n == 1:
        #  current_index = end_index + 1
        #else:
        #  current_index = end_index - (len(compared_text.split(' ', 1)[1]) + 1)

        #Remove ids, check if compared_text is in the dict and use the dict key val. If dict val coatins more than one entries
        #write a uniques paragraph identification string in some file for processing later
        result_detection['entities'].append(
            {
                #"id": str(uuid4()),
                "entity": compared_text,
                "similar_with": similar_with,
                "category": similarity_score[-1],
                "score": similarity_score[1],
                "index": {
                    "start": start_index,
                    "end": end_index
                }
            }
        )
      current_index += len(tokens[i]) + 1 #an approximate calculation, will be behind the actual current index
  result_detection['entities'] = sorted(result_detection['entities'], key=lambda x: x['index']['start'])
  result_detection = handle_slicing(result_detection)
  #result_detection['text'] = text
  return result_detection

### create a new vocabulary and a dictionary

1. loop through each entity in the set "loaded data"
2. If the entity has no stop word, add it to "new_vocab" set without any processing
3. Else, remove all stopwords from the entity. Add the cleaned entity in new_vocab.
4. If the new entity's len<=3 OR it is in the list of common words but not in list of important words, then ignore it
5. Else, add it as part of a dictionary named "vocab_change_dict" whose keys are cleaned entities (without any stopword), and value is a LIST of corresponding original entity.

For final NER using lexifuzz, we will use the vocab_list as the set new_vocab. If the identified entity is a key of dictionatry having len(value)>1, we will store the paragraph's info seperately so that it can be processed later.

In [214]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# loaded_DUMMY_data = ["nerves of brain", "nerves in brain","nerves of spinal cord", "is_a", "created", 
#                "fibers", "is a", "the cat"] 

new_vocab = set()
vocab_change_dict = dict()
for ent in loaded_data:
    word_list = ent.split()
    stops = [w for w in word_list if w in stop_words]
    if len(stops) > 0: # if an entity in loaded data indeed has a stop word
        word_list_no_stop = [w for w in word_list if w not in stop_words]
        new_ent = ' '.join(word_list_no_stop)
        #print(ent, "->", new_ent)
        #break
        if len(new_ent) <= 3 or ((new_ent in word_set) and (new_ent not in loaded_nltk_words_cleaned)) :
            #print(f"Skipping entity {ent} as new entity is small or common words")
            continue
        if new_ent in vocab_change_dict:
            existing_vals = vocab_change_dict[new_ent]
            existing_vals.append(ent)
            vocab_change_dict[new_ent] = existing_vals
        else:
            vocab_change_dict[new_ent] = [ent]
            new_vocab.add(new_ent)
        
    else: # meaning there are no stopwords in the entity, then it is added directly in the new_vocab
        new_vocab.add(ent)

  #if len(stop_list) > 0:
    #    print(ent)
# With REFERENCE TO DUMMY DATA: 
# new_vocab={'created', 'fibers', 'is_a', 'nerves brain', 'nerves spinal cord'}
# vocab_change_dict = {'nerves brain': ['nerves of brain', 'nerves in brain'],
#                     'nerves spinal cord': ['nerves of spinal cord']}

In [215]:
ambiguous_entities = [k for k,v in vocab_change_dict.items() if len(v)>1]
non_ambiguous_entities = [k for k,v in vocab_change_dict.items() if len(v)==1]

In [216]:
# ambiguous_entities[:5]
# ['regulation histone h3-k79 dimethylation',
#  'malignant neoplasm cervical esophagus',
#  'neuropeptide receptor',
#  'negative regulation lymphocyte mediated immunity',
#  'hemarthrosis shoulder region']
# [v for v in new_vocab if '(+)-kotanin anabolism' in v] #'negative regulation (+)-kotanin anabolism' is there
# len([v for v in new_vocab if len(v.split())>25][0].split())


# p=6
# q=25
# to_be_discarded = [v for v in new_vocab if p<=len(v.split())<30]
# for k in to_be_discarded:
#     try:
#         vocab_change_dict.pop(k)
#     except:
#         pass
#     new_vocab.discard(k)

In [217]:
# REMOVE BIG ENTITY
to_be_discarded = [v for v in new_vocab if 28<=len(v.split())]
for k in to_be_discarded:
    try:
        vocab_change_dict.pop(k)
    except:
        pass
    new_vocab.discard(k)

In [218]:
neuro_vocab_list = list(new_vocab)

dictionary = {
    'neuro_entities' : neuro_vocab_list
}

### JSON generators here

#### process_txt_files & process_subfolder

In [219]:
def process_txt_files(folder_path, lock): 
    entity_list = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Processing the txt file {file_path}\n")
            with open(file_path, 'r', encoding='utf-8') as file:
                try:
                    content = file.read()
                    entities = get_entities_from_lexifuzz(content) 
                    entity_list[file_name.replace('.txt', '')] = entities
                except Exception as e:
                    print(f"SOME ERROR = {e} for {file_path}")
                    with lock:
                        error_list.append((file_path, str(e)))
                    return []
    return entity_list

def process_subfolder(subfolder_path, lock): 
    subfolder_dict = {}
    for subfolder_name in os.listdir(subfolder_path): 
        subfolder_dir = os.path.join(subfolder_path, subfolder_name)
        if os.path.isdir(subfolder_dir):
            subfolder_dict[subfolder_name] = process_txt_files(subfolder_dir, lock)
    return subfolder_dict

#### generate_dictionaries_via_root

In [220]:
# def generate_dictionaries_via_root(root_folder): #root_folder=ROOT
#     for folder_name in os.listdir(root_folder):
#         try:
#             start = time.time()
#             folder_path = os.path.join(root_folder, folder_name)
#             if os.path.isdir(folder_path):
#                 subfolder_dict = {}
#                 abstract_folder = os.path.join(folder_path, 'Abstract')
#                 body_folder = os.path.join(folder_path, 'Body')
    
#                 if os.path.isdir(abstract_folder):
#                     subfolder_dict['Abstract'] = process_txt_files(abstract_folder)
    
#                 if os.path.isdir(body_folder):
#                     subfolder_dict['Body'] = process_subfolder(body_folder)
    
#                 # Save dict as JSON
#                 output_file = os.path.join(folder_path, f"{folder_name}.json")
#                 with open(output_file, 'w', encoding='utf-8') as json_file:
#                     json.dump(subfolder_dict, json_file, indent=4)
#             end = time.time()
#             duration = round(end-start,2)
#             print(f"It took {duration} seconds for the folder {folder_name} ")
#         except Exception as e:
#             erronous_paragraphs[folder_name]=e

#     print("ERROR DICTIONARY IS:\n",erronous_paragraphs)
#     with open('error_id_for_NER.txt','w') as f: # Write all error_ids in a seperate txt file for later analysis
#         for key in erronous_paragraphs.keys():
#             f.write(key+'\n')

#### generate_dictionaries_via_papers

In [221]:
def generate_dictionaries_via_papers(folder_path, lock):
    folder_name = os.path.basename(folder_path)  
    try:
        start = time.time()
        print(f"Starting process for {folder_path} at time={start}\n")
        if os.path.isdir(folder_path):
            subfolder_dict = {}
            abstract_folder = os.path.join(folder_path, 'Abstract')
            body_folder = os.path.join(folder_path, 'Body')

            if os.path.isdir(abstract_folder):
                subfolder_dict['Abstract'] = process_txt_files(abstract_folder, lock)

            if os.path.isdir(body_folder):
                subfolder_dict['Body'] = process_subfolder(body_folder, lock)

            # Save dict as JSON
            output_file = os.path.join(folder_path, f"{folder_name}.json")
            with open(output_file, 'w', encoding='utf-8') as json_file:
                json.dump(subfolder_dict, json_file, indent=4)
        error = None
    except Exception as e:
        print(f"SOME ERROR = {e} for {folder_name}")
        error = (folder_name, str(e))
        with lock:
            error_list.append((folder_path, str(e)))
    finally:
        end = time.time()
        duration = round(end - start, 2)
        print(f"It took {duration} seconds for the folder {folder_name}")
        with lock:
            with open("logger_multiprocessor_5.txt","a") as file:
                file.write(f"{folder_name}:{duration}\n")
        return error

### Data for processing

In [222]:
# import os
# folder_path_list = os.listdir("../dummy_JSON_root")
# folder_path_list = [path for path in folder_path_list if path!='.ipynb_checkpoints']
# folder_path_list = [f'../dummy_JSON_root/{path}' for path in folder_path_list]
# folder_path_list

In [223]:
# with open('neuro_IDs_for_neuroscience.txt', 'r') as f:
#     ids = f.readlines()
#     ids = [k.strip() for k in ids]
#     ids_to_be_processed = ids[20:120]
# ids_to_be_processed = get_paper_ids_to_process(400)

In [224]:
# # GETTING RELEVANT PAPER IDs and PATHS FOR PROCESSING
# ids_to_be_processed = get_paper_ids_to_process(1000)
# folder_path_list = [f'../OUTPUT_FOLDER/{ID}' for ID in ids_to_be_processed]
# ids_to_be_processed[:5]
set1, set2 = set(),set()
with open("neuro_IDs_for_neuroscience.txt") as f:
    list1 = [k.strip() for k in f.readlines()]
    set1 = set(list1)
# len(list1) #3548

with open("NER_papers_processed.txt") as f:
    list2 = [k.strip() for k in f.readlines()]
    set2 = set(list2)
len(list2) #3548

2620

In [225]:
ids_to_be_processed = list(set1.difference(set2))
len(ids_to_be_processed) #928
folder_path_list = [f'../OUTPUT_FOLDER/{ID}' for ID in ids_to_be_processed]
ids_to_be_processed[:5]

['2492885', '2726679', '6674504', '1569502', '8337299']

In [226]:
len(folder_path_list), folder_path_list[:10]

(928,
 ['../OUTPUT_FOLDER/2492885',
  '../OUTPUT_FOLDER/2726679',
  '../OUTPUT_FOLDER/6674504',
  '../OUTPUT_FOLDER/1569502',
  '../OUTPUT_FOLDER/8337299',
  '../OUTPUT_FOLDER/1331416',
  '../OUTPUT_FOLDER/2074932',
  '../OUTPUT_FOLDER/509312',
  '../OUTPUT_FOLDER/1397826',
  '../OUTPUT_FOLDER/2235197'])

In [227]:
# folder_path_list = [f'../OUTPUT_FOLDER/{ID}' for ID in ids_to_be_processed]
# folder_path_list[:3]

### Multiprocessing

In [228]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
import os, time

In [229]:
def worker_wrapper(folder_path, lock):
    return generate_dictionaries_via_papers(folder_path, lock)

def multi_thread_call(max_workers = 50, folder_path_list=None):
    with multiprocessing.Manager() as manager:
        lock = manager.Lock()
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(worker_wrapper, path, lock) for path in folder_path_list]
            for future in as_completed(futures):
                future.result()
        return list(error_list)

### MAIN FUNCTION

In [ ]:
if __name__ == "__main__":
    # Global list to store errors
    error_list = multiprocessing.Manager().list()

    # STARTING THE PROCESSING
    start = time.time()
    final_error_list = multi_thread_call(max_workers=100, folder_path_list=folder_path_list)
    end = time.time()
    duration = round(end - start, 2)
    print(f"Processing completed in {duration} seconds")
    
    
    # Print the errors
    if final_error_list:
        print("Errors encountered:")
        for path, error in final_error_list:
            print(f"Path: {path}, Error: {error}")
    else:
        print("No errors encountered.")
        with open("NER_papers_processed.txt",'a') as file:
            for ID in ids_to_be_processed:
                file.write(ID+"\n")

Starting process for ../OUTPUT_FOLDER/2492885 at time=1725203688.8384247

Processing the txt file ../OUTPUT_FOLDER/2492885/Abstract/paragraph_1.txt

Starting process for ../OUTPUT_FOLDER/1331416 at time=1725203688.8598187
Starting process for ../OUTPUT_FOLDER/2074932 at time=1725203688.8574495
Starting process for ../OUTPUT_FOLDER/5901350 at time=1725203688.8623145
Starting process for ../OUTPUT_FOLDER/6674504 at time=1725203688.8485782
Starting process for ../OUTPUT_FOLDER/1397826 at time=1725203688.858706
Starting process for ../OUTPUT_FOLDER/1015135 at time=1725203688.867616
Starting process for ../OUTPUT_FOLDER/2834928 at time=1725203688.8612075
Starting process for ../OUTPUT_FOLDER/1569502 at time=1725203688.855699
Starting process for ../OUTPUT_FOLDER/2726679 at time=1725203688.8462825
Starting process for ../OUTPUT_FOLDER/8337299 at time=1725203688.8597512
Starting process for ../OUTPUT_FOLDER/509312 at time=1725203688.859151
Starting process for ../OUTPUT_FOLDER/1018176 at time

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '–']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '–']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '—']


Processing the txt file ../OUTPUT_FOLDER/2408653/Abstract/paragraph_1.txt



Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '–']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '–']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '=']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '=']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '=']


Processing the txt file ../OUTPUT_FOLDER/300687/Body/Teaming Up/paragraph_7.txt



...

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


Processing the txt file ../OUTPUT_FOLDER/2879381/Body/General/paragraph_9.txt

Processing the txt file ../OUTPUT_FOLDER/2798959/Body/3 Results/paragraph_6.txt



Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
Applied processor reduces input query to empty string, all comparisons will have s

It took 783.81 seconds for the folder 2658651
Starting process for ../OUTPUT_FOLDER/1618511 at time=1725208044.957073

Processing the txt file ../OUTPUT_FOLDER/1618511/Abstract/paragraph_1.txt



Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


In [76]:
print(f"Error List = {final_error_list}")

Error List = []


### COPY/CUT PROCESSED FILES

In [116]:
# with open("NER_papers_processed.txt",'r') as f:
#     ids_processed_till_now = [k.strip() for k in f.readlines()]

# folder_path_list = [f"../OUTPUT_FOLDER/{id}/{id}.json" for id in ids_processed_till_now]
# folder_path_list[:5]

['../OUTPUT_FOLDER/8742975/8742975.json',
 '../OUTPUT_FOLDER/8485740/8485740.json',
 '../OUTPUT_FOLDER/10790504/10790504.json',
 '../OUTPUT_FOLDER/10777891/10777891.json',
 '../OUTPUT_FOLDER/10760685/10760685.json']

In [118]:
# from subprocess import call

# def cp_dir(source, target):
#     call(['cp', '-a', source, target]) # Linux

# def mv_dir(source, target):
#     call(['mv', source, target])  # Linux

# to_directory  = "./ALL_JSON_files"

# for folder in folder_path_list:
#     from_directory = folder
#     # copy_tree(source, destination)
#     try:
#         cp_dir(from_directory, to_directory)
#         # mv_dir(from_directory, to_directory)
#     except Exception as e:
#         print(f"Error = {e} for folder={folder}")

### Experiments using new_vocab and new_dictionary

#### Case 1: entities which are NOT in vocab_change_dict.keys()

In [223]:
# SCRATCH CELL FOR EXPERIMENTATION
# vocab_change_dict['nerve spine']  # >>['nerve of spine']
# vocab_change_dict['regulation cellular defense response']
#vocab_change_dict['regulation histone h3-k79 dimethylation'] # >>['down regulation of histone h3-k79 dimethylation', 'up regulation of histone h3-k79 dimethylation']
# 'nerve spine' in new_vocab, 'negative regulation (+)-kotanin anabolism' in new_vocab, 'regulation histone h3-k79 dimethylation' in new_vocab

# vocab_change_dict = dict()
# vocab_change_dict['nerve spine']=['nerve of spine']
# vocab_change_dict['regulation translational elongation']=['regulation of translational elongation','down regulation of translational elongation','up regulation of translational elongation']
# vocab_change_dict['regulation cellular defense response'] = ['down regulation of cellular defense response','up regulation of cellular defense response','regulation of cellular defense response']
# vocab_change_dict['negative regulation (+)-kotanin anabolism']=['negative regulation of (+)-kotanin anabolism']
# vocab_change_dict['negative regulation hr']=['negative regulation of hr']
# # vocab_change_dict['negative regulation kotanin anabolism']=['negative regulation of kotanin anabolism']

# new_vocab = {'negative regulation hr', 'negative regulation (+)-kotanin anabolism', 'nerve spine','regulation translational elongation','regulation cellular defense response', 'tissue','cervical'}
# # new_vocab = {'negative regulation kotanin anabolism', 'nerve spine','regulation translational elongation','regulation cellular defense response', 'tissue','cervical'}


In [244]:
neuro_vocab_list = list(new_vocab)

dictionary = {
    'neuro_entities' : neuro_vocab_list
}

In [247]:
start = time.time()
# text = "In recent years, the field of neuroscience has gone through rapid experimental advances and a significant increase in the use of quantitative and computational methods. This growth has created a need for clearer analyses of the theory and modeling approaches used in the field. This issue is particularly complex in neuroscience because the field studies phenomena that cross a wide range of scales and often require consideration at varying degrees of abstraction, from precise biophysical interactions to the computations they implement. We argue that a pragmatic perspective of science, in which descriptive, mechanistic, and normative models and theories each play a distinct role in defining and bridging levels of abstraction, will facilitate neuroscientific practice. This analysis leads to methodological suggestions, including selecting a level of abstraction that is appropriate for a given problem, identifying transfer functions to connect models and data, and the use of models themselves as a form of experiment."
# text="nerves of spine and down regulation of translational elongations along with up regulation of cellular defense response 
# text = 'Having the negative regulation of kotanin anabolism'
text = "Do you know that nerves of spine below cervical are complex in nature"
result = get_entity_df(text)
end=time.time()
duration = round(end-start,2)
print(f"Processing completed in {duration} seconds")
result

Skip verb Do
Processing completed in 3.45 seconds


,entity,similar_with,category,score,index
0,nerves of spine,nerve of spine,neuro_entities,96,"{'start': 17, 'end': 31}"


#### Case 2: non-ambiguous entities which are in vocab_change_dict.keys()


In [248]:
print(f"'negative regulation (+)-kotanin anabolism'===>{vocab_change_dict['negative regulation (+)-kotanin anabolism']}")
# Hence: 'negative regulation (+)-kotanin anabolism' is a non-ambiguous entity

'negative regulation (+)-kotanin anabolism'===>['negative regulation of (+)-kotanin anabolism']


In [250]:
neuro_vocab_list = list(new_vocab)

dictionary = {
    'neuro_entities' : neuro_vocab_list
}
text = 'negative regulation of (+)-kotanin anabolism' # this is actual entity whose key is:
# 'negative regulation (+)-kotanin anabolism'
result = get_entity_df(text)
result

,entity,similar_with,category,score,index
0,negative regulation of,negative regulation of hr,neuro_entities,93,"{'start': 0, 'end': 21}"
1,kotanin anabolism,(+)-kotanin anabolism,neuro_entities,100,"{'start': 27, 'end': 43}"


In [214]:
'negative regulation hr' in new_vocab, vocab_change_dict['negative regulation hr']

(True, ['negative regulation of hr'])

In [253]:
'(+)-kotanin anabolism' in new_vocab # '(+)-kotanin anabolism' is only in set, not in dictionary

(True,)

In [258]:
'negative regulation (+)-kotanin anabolism' in new_vocab, \
vocab_change_dict['negative regulation (+)-kotanin anabolism']

(True, ['negative regulation of (+)-kotanin anabolism'])

#### Case 3: entities in text having stop words and are ambiguous

In [266]:
# text = 'Regarding regulation histone h3-k79 dimethylation, I am currently studying two \
# topics: up regulation of histone h3-k79 dimethylation,\
# and down regulation of histone h3-k79 dimethylation.'
text = 'regulation histone h3-k79 dimethylation'
# NOTE: 'regulation histone h3-k79 dimethylation' is an ambiguous entity mapped to:
# up regulation of histone h3-k79 dimethylation, down regulation of histone h3-k79 dimethylation
df = get_entity_df(text)
df

,entity,similar_with,category,score,index
0,histone h3-k79 dimethylation,histone h3 k79 dimethylation,neuro_entities,100,"{'start': 11, 'end': 38}"


In [264]:
# 'histone h3 k79 dimethylation' in new_vocab, vocab_change_dict.get('histone h3 k79 dimethylation')
# (True, None)

(True, None)

In [265]:
# 'regulation histone h3-k79 dimethylation' in new_vocab, vocab_change_dict.get('regulation histone h3-k79 dimethylation')
# (True,
#  ['down regulation of histone h3-k79 dimethylation',
#   'up regulation of histone h3-k79 dimethylation'])

(True,
 ['down regulation of histone h3-k79 dimethylation',
  'up regulation of histone h3-k79 dimethylation'])